In [6]:
!git clone -b develop https://github.com/made-kdd-2021/image-cls.git
%cd /content/image-cls
!pip install -r ./requirements.cpu.txt -r ./requirements.dev.txt

Cloning into 'image-cls'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 115 (delta 29), reused 102 (delta 25), pack-reused 0
Receiving objects: 100% (115/115), 27.51 KiB | 1.45 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/image-cls
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 15.7 MB 20 kB/s 
     |████████████████████████████████| 175.4 MB 27 kB/s 
     |████████████████████████████████| 45 kB 1.6 MB/s 
     |████████████████████████████████| 392 kB 9.2 MB/s 
     |████████████████████████████████| 393 kB 38.6 MB/s 
     |████████████████████████████████| 212 kB 60.0 MB/s 
     |████████████████████████████████| 48 kB 5.0 MB/s 
     |████████████████████████████████| 125 kB 54.3 MB/s 
     |████████████████████████████████| 41 kB 38 kB/s 
     |████████████████████████████████| 180 kB 53.1 MB/s 
     |

In [7]:
!ls /content/


drive  image-cls  sample_data


In [4]:
%cd /content/image-cls
%%writefile ./.dvc/config.local

['remote "storage"']
    gdrive_service_account_json_file_path = /content/drive/file_name.json

Writing ./.dvc/config.local


In [5]:
!dvc pull

Transferring:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
!
  0%|          |296809bf91fe1dc233eee9e1f038f6     0.00/? [00:00<?,        ?B/s]
296809bf91fe1dc233eee9e1f038f6:   0% 0.00/1.15G [00:00<?, ?B/s{'info': ''}]     
  8% 100M/1.15G [00:01<00:19, 57.5MB/s{'info': ''}]                        
 17% 200M/1.15G [00:04<00:20, 49.8MB/s{'info': ''}]
 25% 300M/1.15G [00:05<00:15, 58.9MB/s{'info': ''}]
 34% 400M/1.15G [00:06<00:11, 69.9MB/s{'info': ''}]
 42% 500M/1.15G [00:07<00:09, 76.4MB/s{'info': ''}]
 51% 600M/1.15G [00:09<00:08, 75.0MB/s{'info': ''}]
 59% 700M/1.15G [00:10<00:07, 68.7MB/s{'info': ''}]
 68% 800M/1.15G [00:12<00:05, 73.7MB/s{'info': ''}]
 76% 900M/1.15G [00:13<00:03, 77.9MB/s{'info': ''}]
 85% 0.98G/1.15G [00:15<00:02, 68.1MB/s{'info': ''}]
 93% 1.07G/1.15G [00:16<00:01, 74.8MB/s{'info': ''}]
100% 1.15G/1.15G [00:17<00:00, 80.7MB/s{'info': ''}]
Computing md5 for a large file '/content/image-cls/.dvc/cache/cf/296809bf91fe1dc233eee9e1f038f6'. This is only done once.
Transferr

In [6]:
import os
import pathlib
import numpy as np
import zipfile
import numbers
import torch
from torch import nn
from torch import utils
from torchvision import io
from torchvision import transforms
import torchvision.transforms.functional as F
from torchvision.transforms.functional import pad
from torch.utils.data import DataLoader, random_split

In [7]:
def get_padding(image):    
    w, h = image.size
    max_wh = np.max([w, h])
    h_padding = (max_wh - w) / 2
    v_padding = (max_wh - h) / 2
    l_pad = h_padding if h_padding % 1 == 0 else h_padding+0.5
    t_pad = v_padding if v_padding % 1 == 0 else v_padding+0.5
    r_pad = h_padding if h_padding % 1 == 0 else h_padding-0.5
    b_pad = v_padding if v_padding % 1 == 0 else v_padding-0.5
    padding = (int(l_pad), int(t_pad), int(r_pad), int(b_pad))
    return padding

class NewPad(object):
    def __init__(self, fill=0, padding_mode='constant'):
        assert isinstance(fill, (numbers.Number, str, tuple))
        assert padding_mode in ['constant', 'edge', 'reflect', 'symmetric']

        self.fill = fill
        self.padding_mode = padding_mode
        
    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to be padded.

        Returns:
            PIL Image: Padded image.
        """
        return F.pad(img, get_padding(img), self.fill, self.padding_mode)
    
    def __repr__(self):
        return self.__class__.__name__ + '(padding={0}, fill={1}, padding_mode={2})'.\
            format(self.fill, self.padding_mode)

In [8]:
CONSTANT_MEAN = [0.485, 0.456, 0.406]
CONSTANT_STD = [0.229, 0.224, 0.225]
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip()
    ]),
    'general': transforms.Compose([
        #NewPad(),
        transforms.Resize(size=500),
        transforms.ConvertImageDtype(dtype=torch.float32), 
        transforms.Grayscale(num_output_channels=3),
        #transforms.ToTensor(),
        transforms.Normalize(mean=CONSTANT_MEAN, std=CONSTANT_STD)
    ]),
}

In [15]:
class ChestXray(utils.data.Dataset):
    def __init__(self, path_to_zip, set_name, class_names = {"NORMAL": 0, "PNEUMONIA": 1}):
        self._path_to_zip = path_to_zip
        self._class_mapping = class_names
        self._image_lists = []
        self._image_classes = []
        self._set_name = set_name
        assert set_name in ['train', 'test', 'val']
        if set_name == 'train':
          self._transform = transforms.Compose([data_transforms['general'], data_transforms['train']])
        elif set_name == 'test' or set_name == 'val':
          self._transform = transforms.Compose([data_transforms['general']]);

        
        with zipfile.ZipFile(path_to_zip) as zip_file:
            for zip_info in zip_file.infolist():
                if not zip_info.is_dir():
                    image_path = pathlib.Path(zip_info.filename)
                    if self._set_name == image_path.parent.parent.name:
                        self._image_classes.append(self._class_mapping[image_path.parent.name])
                        self._image_lists.append(zip_info)

    def __len__(self):
        return len(self._image_lists)
    
    def __getitem__(self, index):
        with zipfile.ZipFile(self._path_to_zip) as zip_file:
            with zip_file.open(self._image_lists[index], "r") as image_file:
                image = np.frombuffer(image_file.read(), dtype=np.uint8)
                image = io.decode_jpeg(torch.from_numpy(image.copy()), mode=io.ImageReadMode.RGB)

        return {"image": self._transform(image), "label": self._image_classes[index]}



In [16]:
train_dataset = ChestXray("/content/image-cls/data/raw/chest_xray.zip", "train")
test_dataset = ChestXray("/content/image-cls/data/raw/chest_xray.zip", "test")
val_dataset = ChestXray("/content/image-cls/data/raw/chest_xray.zip", "val")

In [17]:
train_dataset[0]["image"].shape

torch.Size([3, 500, 562])

In [18]:
test_dataset[0]["image"].shape

torch.Size([3, 500, 705])

In [19]:
val_dataset[0]["image"].shape

torch.Size([3, 500, 627])

In [21]:
dataloader = utils.data.DataLoader(test_dataset, 
                                   batch_size=64, 
                                   pin_memory=True, 
                                   shuffle=False, 
                                   drop_last=True, 
                                   num_workers=2)

In [25]:
class XrayDataLoader(utils.data.DataLoader):
    def __init__(self, path_to_zip, set_name, batch_size=64,
                 pin_memory=True, shuffle=False, drop_last=True, num_workers=1):
        self._path_to_zip = path_to_zip
        self._set_name = set_name
        self._batch_size = batch_size
        self._pin_memory = pin_memory
        #self._shuffle = shuffle
        self._drop_last = drop_last
        self._num_workers = num_workers

        assert set_name in ["train", "test", "val"]
        if set_name == 'train':
          self._shuffle = False
        elif set_name == 'test' or set_name == 'val':
          self._shuffle = True;

    def __getitem__(self):
        return utils.data.DataLoader(ChestXray(path_to_zip, set_name), batch_size=self._batch_size, pin_memory = self._pin_memory,
                              shuffle = self._shuffle, drop_last = self._drop_last, num_workers = self._num_workers)    

In [26]:
dataloader = XrayDataLoader("/content/image-cls/data/raw/chest_xray.zip", "test", batch_size=64, 
                                   pin_memory=True, 
                                   shuffle=False, 
                                   drop_last=True, 
                                   num_workers=2)

# New Section